In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data as Data
from sklearn import metrics

In [ ]:
def get_auroc(preds, obs):
        fpr, tpr, thresholds  = metrics.roc_curve(obs, preds, drop_intermediate=False)
        auroc = metrics.auc(fpr, tpr)
        return auroc

def get_auprc(preds, obs):
        precision, recall, thresholds  = metrics.precision_recall_curve(obs, preds)
        auprc = metrics.auc(recall, precision)
        return auprc

def get_aurocs_and_auprs(tpreds, tobs):
        tpreds_df = pd.DataFrame(tpreds)
        tobs_df = pd.DataFrame(tobs)
        task_list = []
        auroc_list = []
        auprc_list = []
        for task in tpreds_df:
                pred = tpreds_df[task]
                obs = tobs_df[task]
                auroc=round(get_auroc(pred,obs),5)
                auprc = round(get_aupr(pred,obs),5)
                task_list.append(task)
                auroc_list.append(auroc)
                auprc_list.append(aupr)
        return auroc_list, auprc_list

In [ ]:
path = './'
challenge_specific_name = 'k562'
train_x = torch.FloatTensor(np.load(path + challenge_specific_name + '_train_seq.npy'))
train_y = torch.FloatTensor(np.load(path + challenge_specific_name + '_train_label.npy'))
valid_x = torch.FloatTensor(np.load(path + challenge_specific_name + '_valid_seq.npy'))
valid_y = torch.FloatTensor(np.load(path + challenge_specific_name + '_valid_label.npy'))
test_x = torch.FloatTensor(np.load(path + challenge_specific_name + '_test_seq.npy'))
test_y = torch.FloatTensor(np.load(path + challenge_specific_name + '_test_label.npy'))

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
loss_func = nn.BCEWithLogitsLoss()
train_loader = Data.DataLoader(dataset=Data.TensorDataset(train_x, train_y), shuffle=True, batch_size = 64)
valid_loader = Data.DataLoader(dataset=Data.TensorDataset(valid_x, valid_y), shuffle=True, batch_size = 64)
test_loader = Data.DataLoader(dataset=Data.TensorDataset(test_x, test_y), shuffle=False, batch_size = 64)


In [ ]:
train_losses = []
valid_losses = []
for epoch in range(10):
    model.train()
    train_loss = 0
    for step, (train_batch_x, train_batch_y) in enumerate(train_loader):
        train_batch_x = train_batch_x.cuda()
        train_batch_y = train_batch_y.cuda()
        out = model(train_batch_x)
        loss = loss_func(out, train_batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        train_batch_x = torch.FloatTensor(np.flip(np.flip(train_batch_x.numpy(), 1), 2)).cuda()
        train_batch_y = train_batch_y.cuda()
        out = model(train_batch_x)
        loss = loss_func(out, train_batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    model.eval()
    for valid_step, (valid_batch_x, valid_batch_y) in enumerate(valid_loader):
        valid_batch_x = valid_batch_x.cuda()
        valid_batch_y = valid_batch_y.cuda()
        val_out = model(valid_batch_x)
        val_loss = loss_func(val_out, valid_batch_y)
        valid_losses.append(val_loss.item())
        valid_batch_x = torch.FloatTensor(np.flip(np.flip(valid_batch_x.numpy(), 1), 2)).cuda()
        valid_batch_y = valid_batch_y.cuda()
        val_out = model(valid_batch_x)
        val_loss = loss_func(val_out, valid_batch_y)
        valid_losses.append(val_loss.item())

In [ ]:
model.eval()
for step, (seq, label) in enumerate(test_loader):
    seq = seq.cuda()
    label = label.cuda()
    test_output = model(seq) 
    for i, j in zip(range(step*64, (step+1)*64),range(0, 64)):
        pred_y[i, :] = test_output.numpy()[j, :]

auroc, auprc = get_auroc_and_auprc(pred, label)